In [1]:
import dlt
import duckdb
from dlt.sources.helpers import requests

In [4]:
# generator function
def loader():
  url = (
    "https://api.github.com/repos/dlt-hub/dlt/issues"
    "?per_page=100&sort=created&directions=desc&state=open"
  )

  while True:
    response = requests.get(url)
    response.raise_for_status()
    yield response.json()

    if "next" not in response.links:
      break
    url = response.links["next"]["url"]


In [3]:
pipeline = dlt.pipeline(
  pipeline_name='paginated-dlt-issues',
  destination='duckdb',
  dataset_name='dlt'
)

In [5]:
print(pipeline.run(loader, table_name='issue', write_disposition='replace'))

Pipeline paginated-dlt-issues load step completed in 0.74 seconds
1 load package(s) were loaded to destination duckdb and into dataset dlthub
The duckdb destination used duckdb:////Users/Kamal/WorkSpace/llm-zoomcamp/Workshops/dlt/paginated-dlt-issues.duckdb location to store data
Load package 1720974260.653474 is LOADED and contains no failed jobs


In [6]:
db = duckdb.connect('./paginated-dlt-issues.duckdb')
db.sql('select count(*) from dlthub.issue').show()  # open issues no. + pull requests 
db.close()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          164 │
└──────────────┘

